### Introduction
이번 챕터에서는 **rnn 모델을 활용하여 multiclass classification task를 해결**합니다.  
사용할 데이터셋은 AG_NEWS입니다. 뉴스 기사를 보고 4개의 토픽('World', 'Sports', 'Business', 'Sci/Tech') 중 적절한 한개로 분류하여야합니다.


In [ ]:
import urllib
import os
import collections
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
import torchtext
from torch.utils.data import Dataset, DataLoader, random_split

### Preparing dataset

In [ ]:
os.makedirs('data', exist_ok=True)
dataset_train, dataset_test = torchtext.datasets.AG_NEWS(root='./data')
tokenizer = torchtext.data.utils.get_tokenizer('basic_english', language="en")
dataset_train = list(dataset_train)
dataset_test = list(dataset_test)

In [ ]:
tokenized_data = []
label_data = []
counter = collections.Counter()

print('Start Tokenizing...')
for label, line in tqdm(dataset_train):
    tokens = tokenizer(line)
    tokenized_data.append(tokens)
    label_data.append(label-1)

print('Making Vocab...')
for line in tqdm(tokenized_data):
    counter.update(line)

vocab = torchtext.vocab.vocab(counter, min_freq=1, specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
class AGNewsDataset(Dataset):
    def __init__(self, tokenized_data, label_data, max_seq = 256):
        self.x = []
        self.y = []
        self.classes = ['World', 'Sports', 'Business', 'Sci/Tech']
        
        for tokens in tqdm(tokenized_data):
            token_ids = [vocab[token] for token in tokens]
            self.x.append(token_ids)
        self.y = label_data
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]) , torch.tensor(self.y[idx])

In [ ]:
news_dataset = AGNewsDataset(tokenized_data, label_data, max_seq=256)

In [ ]:
train_ratio = 0.8
train_size = int(train_ratio*len(news_dataset))
valid_size = len(news_dataset) - train_size
train_dataset, valid_dataset = random_split(news_dataset, [train_size, valid_size])

### Making Model

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, batch_first=True)
        self.out =  nn.Linear(hidden_size, output_size)

    def forward(self, input):
        input = self.embedding(input)
        #input: [batch_size, sentence_len, emb_size]
        output, _ = self.rnn(input)
        #output: [batch_size, sentence_len, hidden_size]
        output = self.out(output)
        #output: [batch_size, sentence_len, output_size]
        output = output.transpose(1,0)
        #output: [sentence_len, batch_size, output_size]
        output = output[-1]
        #output: [batch_size, output_size]

        return output



In [ ]:
model = RNN(vocab_size=len(vocab), embedding_size=64, hidden_size=32, output_size=len(news_dataset.classes))

### Preparing for training

In [ ]:
batch_size=256
learning_rate = 0.001
num_epochs = 10
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def pad_collate_fn(batch):
    collate_x = []
    collate_y = []
    for data, label in batch:
        collate_x.append(data)
        collate_y.append(label)
    collate_x = nn.utils.rnn.pad_sequence(collate_x, padding_value=vocab['<pad>'], batch_first=True)
    collate_y = torch.stack(collate_y)
    return (collate_x, collate_y)



In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= pad_collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn= pad_collate_fn)

In [ ]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
def accuracy(output, label):
    pred = torch.argmax(output, dim=-1)
    acc = torch.sum(pred==label)
    return acc
    

### Training

In [ ]:
def train(dataloader, epoch):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    data_num = 0
    for data,label in tqdm(dataloader, desc=f"Epoch {epoch}"):
        data, label = data.to(device), label.to(device)
        output = model(data)
        optimizer.zero_grad()
        loss = criterion(output, label)
        acc = accuracy(output, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        data_num += data.size(0)

    print(f"Train Epoch: {epoch}, Loss: {epoch_loss/len(dataloader)}, Acc: {epoch_acc/data_num}")


In [ ]:
def evaluate(dataloader, epoch):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    data_num = 0
    with torch.no_grad():
        for data,label in tqdm(dataloader, desc=f"Epoch {epoch}"):
            data, label = data.to(device), label.to(device)
            output = model(data)
            loss = criterion(output, label)
            acc = accuracy(output, label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            data_num += data.size(0)

    print(f"Evaluate Epoch: {epoch}, Loss: {epoch_loss/len(dataloader)}, Acc: {epoch_acc/data_num}")
    return epoch_acc/data_num

In [ ]:
total_acc = 0
best_acc = 0
for epoch in range(num_epochs):
    train(train_dataloader, epoch)
    acc = evaluate(valid_dataloader, epoch)
    best_acc = max(acc, best_acc)
    print('-'*50)

print(f"Best Accuracy: {best_acc}")



### Testing Model

In [ ]:
#TBA